# Triplet Loss with Faces -- Fairness Study

## Initialization

In [ ]:
!pip install torch

In [ ]:
!pip install mat73

In [ ]:
!pip install wandb -qqq
import wandb

In [ ]:
!git clone https://github.com/mgornet/CNPEN

### Check device

In [ ]:
!nvidia-smi

### Import Librairies

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from time import perf_counter
from typing import Callable
import itertools
import mat73
import pandas as pd
import re

import sys
import os
import tarfile

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

import torch
from torch import nn, optim
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

import os.path as op
try:
    from urllib.request import urlretrieve
except ImportError:  # Python 2 compat
    from urllib import urlretrieve
    
from sklearn.metrics import confusion_matrix, auc, roc_curve, \
precision_recall_curve, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from scipy import optimize

In [ ]:
%cd ./CNPEN/files/

from triplet import TripletGenerator, TripletLearner, TripletLoss, TripletLossRaw, \
distance, distance_vectors
from builder import create_dataframe, from_tensor_to_numpy, from_numpy_to_tensor, extend_dataframe
from prints import print_img, print_img_from_path, print_img_from_id, \
print_img_from_classid, print_from_gen, print_from_gen2, print_pair, print_hist_loss, \
print_hist_dist, print_hist_dist_zoom, print_img_category, \
print_roc, print_logistic_regression, print_prec_recall
from test_train_loops import training, testing, adaptative_train, compute_distances
from classification import authentification_img, predict, triplet_acc,\
build_df_fairness, triplet_acc_fairness, bootstrap

In [ ]:
!pwd

## Generate Data

### Create dataframe

In [ ]:
URL = "http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz"
FILENAME = "lfw-deepfunneled.tgz"

if not op.exists(FILENAME):
    print('Downloading %s to %s...' % (URL, FILENAME))
    urlretrieve(URL, FILENAME)

if not op.exists("lfw"):
    print('Extracting image files...')
    tar = tarfile.open("lfw-deepfunneled.tgz")
    tar.extractall("lfw")
    tar.close()

In [ ]:
PATH = "lfw/lfw-deepfunneled/"

In [ ]:
tic = perf_counter()
df_init, all_imgs = create_dataframe()
toc = perf_counter()
print(f"DataFrame creation: {((toc - tic)/60):.1f} min")

In [ ]:
tic = perf_counter()
df = extend_dataframe(df_init)
toc = perf_counter()
print(f"DataFrame extention: {((toc - tic)/60):.1f} min")

### Build sets, generators and network

In [ ]:
num_classes = len(df.Classid.unique())
print("Number of individuals: ", num_classes)

In [ ]:
indiv_min = df.Classid.min()
split_train_valid = int(num_classes * 0.75)
split_train_test = int(num_classes * 0.8)
indiv_max = df.Classid.max()

In [ ]:
print(f"Train set from indiv {indiv_min} to {split_train_valid-1}")
print(f"Valid set from indiv {split_train_valid} to {split_train_test-1}")
print(f"Test set from indiv {split_train_test} to {indiv_max}")

In [ ]:
df_train = df[df.Classid<split_train_valid]
df_valid = df[(df.Classid>=split_train_valid)&(df.Classid<split_train_test)]
df_test = df[df.Classid>=split_train_test]

In [ ]:
print("Number of training images: ", len(df_train))
print("Number of validation images: ", len(df_valid))
print("Number of testing images: ", len(df_test))
print("Number of total images: ", len(df_train)+len(df_valid)+len(df_test))
print("len original: ", len(df))

In [ ]:
print("Number of individuals in the training set: ", len(df_train.Classid.unique()))
print("Number of individuals in the validation set: ", len(df_valid.Classid.unique()))
print("Number of individuals in the testing set: ", len(df_test.Classid.unique()))

In [ ]:
value_count = df_train.Classid.value_counts()
print("Number of individuals with more than one image in the training set: ", len(value_count[value_count.values>1]))
value_count = df_valid.Classid.value_counts()
print("Number of individuals with more than one image in the validation set: ", len(value_count[value_count.values>1]))
value_count = df_test.Classid.value_counts()
print("Number of individuals with more than one image in the testing set: ", len(value_count[value_count.values>1]))

In [ ]:
df_valid.head()

In [ ]:
BATCH_SIZE = 128 # 128
BATCH_VALID_SIZE = 128 #128 #8
BATCH_TEST_SIZE = 128 #128 #32

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
margin = 0.2
criterion = TripletLoss(margin)
criterion_test = TripletLossRaw(margin)

### Load Model

In [ ]:
# Load pretrained model

model = TripletLearner(base_channels=32)
model.load_state_dict(torch.load("./models/without_jitter_and_p.pth",map_location=torch.device('cpu')))
model = model.to(device)
model.eval()

### Determine THRESHOLD

In [ ]:
# Build THRESHOLD (for details, see the notebook "determine_threshold")

gen = TripletGenerator(df_valid, all_imgs, BATCH_VALID_SIZE, device, model, margin)
loader = DataLoader(gen, batch_size=None, shuffle=True)

list_loader = []
for _ in range(10):
    list_loader.extend(list(loader))

pos_dist, neg_dist, _ = compute_distances(list_loader, device, model) #loader

y_pos = [1 for _ in range(len(pos_dist))]
y_neg = [0 for _ in range(len(neg_dist))]

y = y_pos + y_neg
X = pos_dist + neg_dist
Xmoins = np.array(X)*(-1)
Xlogistic = np.array(Xmoins).reshape(-1,1)

clf = LogisticRegression(random_state=0).fit(Xlogistic, y)

THRESHOLD = (clf.intercept_/clf.coef_)[0,0]
print("THRESHOLD with logistic regression:", THRESHOLD)

## Fairness study

### Build data fairness

In [ ]:
gen = TripletGenerator(df_test, all_imgs, BATCH_TEST_SIZE, device, model, margin, return_id=True)

In [ ]:
tic = perf_counter()
df_fairness = build_df_fairness(all_imgs, df_test, gen, 20, device, model, THRESHOLD)
toc = perf_counter()
print(f"DataFrame creation: {((toc - tic)/60):.1f} min")

In [ ]:
df_fairness

In [ ]:
df_fairness.describe()

### Look at general stats

In [ ]:
print("Same identity - mean distance: ", df_fairness[df_fairness.y_true==1].Distance.mean())
print("Same identity - std distance: ", df_fairness[df_fairness.y_true==1].Distance.std())
print("Same identity - percentiles: ", np.percentile(df_fairness[df_fairness.y_true==1]['Distance'], [5,25,50,75,95]))
print("\n")
print("Different identities - mean distance: ", df_fairness[df_fairness.y_true==0].Distance.mean())
print("Different identities - std distance: ", df_fairness[df_fairness.y_true==0].Distance.std())
print("Different identities - percentiles: ", np.percentile(df_fairness[df_fairness.y_true==0]['Distance'], [5,25,50,75,95]))
print("\n")
print("Mean accuracy: ", df_fairness['correct_predict'].mean())
print("Bootstrapping mean accuracy: ", bootstrap(df_fairness, agg_func=lambda df: df['correct_predict'].mean()))
print("\n")
print("Triplet accuracy: ", triplet_acc_fairness(df_fairness))

In [ ]:
pos_dist = df_fairness[df_fairness.y_true==1]['Distance']
neg_dist = df_fairness[df_fairness.y_true==0]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
X = -np.array(df_fairness.Distance)
y = np.array(df_fairness.y_true)
y_pred = np.array(df_fairness.y_pred)

In [ ]:
fpr_dist, tpr_dist, thresholds_dist = roc_curve(y, X)
roc_auc_dist = auc(fpr_dist,tpr_dist)

precision_dist, recall_dist, thresholds_recall_dist = precision_recall_curve(y, X)
auc_s_dist = auc(recall_dist, precision_dist)

In [ ]:
print_roc(fpr_dist, tpr_dist, roc_auc_dist)
print_prec_recall(precision_dist, recall_dist, auc_s_dist)

In [ ]:
tp,fp,fn,tn = confusion_matrix(y, y_pred).ravel()

TPR = tp/(tp+fp)
FPR = fp/(tp+fp)
TNR = tn/(tn+fn)
FNR = fn/(tn+fn)

In [ ]:
print("Confusion Matrix Total")
print(confusion_matrix(y, y_pred))

print("\n","Accuracy score:",accuracy_score(y, y_pred))
 
print("\n", "f1 score:", f1_score(y, y_pred), "\n")

print('TPR: ', TPR)
print('FPR: ', FPR)
print('TNR: ', TNR)
print('FNR: ', FNR)

### Stats in subgroups

#### White Male vs Non White Male

In [ ]:
pos_dist = df_fairness[(df_fairness.y_true==1) & (df_fairness.AB_WhiteMale==1)]['Distance']
neg_dist = df_fairness[(df_fairness.y_true==0) & (df_fairness.AB_WhiteMale==1)]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
pos_dist = df_fairness[(df_fairness.y_true==1) & (df_fairness.AB_NoWhiteMale==1)]['Distance']
neg_dist = df_fairness[(df_fairness.y_true==0) & (df_fairness.AB_NoWhiteMale==1)]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
pos_dist = df_fairness[(df_fairness.y_true==1) & (df_fairness.AB_WhiteMale==0)]['Distance']
neg_dist = df_fairness[(df_fairness.y_true==0) & (df_fairness.AB_WhiteMale==0)]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
y_AB_WhiteMale = np.array(df_fairness[df_fairness['AB_WhiteMale']==1].y_true)
y_pred_AB_WhiteMale = np.array(df_fairness[df_fairness['AB_WhiteMale']==1].y_pred)

tp_AB_WhiteMale,fp_AB_WhiteMale,fn_AB_WhiteMale,tn_AB_WhiteMale = confusion_matrix(y_AB_WhiteMale, y_pred_AB_WhiteMale).ravel()

TPR_AB_WhiteMale = tp_AB_WhiteMale/(tp_AB_WhiteMale+fp_AB_WhiteMale)
FPR_AB_WhiteMale = fp_AB_WhiteMale/(tp_AB_WhiteMale+fp_AB_WhiteMale)
TNR_AB_WhiteMale = tn_AB_WhiteMale/(tn_AB_WhiteMale+fn_AB_WhiteMale)
FNR_AB_WhiteMale = fn_AB_WhiteMale/(tn_AB_WhiteMale+fn_AB_WhiteMale)

y_AB_NoWhiteMale = np.array(df_fairness[df_fairness['AB_NoWhiteMale']==1].y_true)
y_pred_AB_NoWhiteMale = np.array(df_fairness[df_fairness['AB_NoWhiteMale']==1].y_pred)

tp_AB_NoWhiteMale,fp_AB_NoWhiteMale,fn_AB_NoWhiteMale,tn_AB_NoWhiteMale = confusion_matrix(y_AB_NoWhiteMale, y_pred_AB_NoWhiteMale).ravel()

TPR_AB_NoWhiteMale = tp_AB_NoWhiteMale/(tp_AB_NoWhiteMale+fp_AB_NoWhiteMale)
FPR_AB_NoWhiteMale = fp_AB_NoWhiteMale/(tp_AB_NoWhiteMale+fp_AB_NoWhiteMale)
TNR_AB_NoWhiteMale = tn_AB_NoWhiteMale/(tn_AB_NoWhiteMale+fn_AB_NoWhiteMale)
FNR_AB_NoWhiteMale = fn_AB_NoWhiteMale/(tn_AB_NoWhiteMale+fn_AB_NoWhiteMale)

y_AoB_NoWhiteMale = np.array(df_fairness[df_fairness['AB_WhiteMale']==0].y_true)
y_pred_AoB_NoWhiteMale = np.array(df_fairness[df_fairness['AB_WhiteMale']==0].y_pred)

tp_AoB_NoWhiteMale,fp_AoB_NoWhiteMale,fn_AoB_NoWhiteMale,tn_AoB_NoWhiteMale = confusion_matrix(y_AoB_NoWhiteMale, y_pred_AoB_NoWhiteMale).ravel()

TPR_AoB_NoWhiteMale = tp_AoB_NoWhiteMale/(tp_AoB_NoWhiteMale+fp_AoB_NoWhiteMale)
FPR_AoB_NoWhiteMale = fp_AoB_NoWhiteMale/(tp_AoB_NoWhiteMale+fp_AoB_NoWhiteMale)
TNR_AoB_NoWhiteMale = tn_AoB_NoWhiteMale/(tn_AoB_NoWhiteMale+fn_AoB_NoWhiteMale)
FNR_AoB_NoWhiteMale = fn_AoB_NoWhiteMale/(tn_AoB_NoWhiteMale+fn_AoB_NoWhiteMale)

In [ ]:
print("A and B White Male - mean accuracy: ", df_fairness[df_fairness['AB_WhiteMale']==1]['correct_predict'].mean())
print("A and B White Male - bootstrapping mean accuracy: ", bootstrap(df_fairness[df_fairness['AB_WhiteMale']==1], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B White Male - triplet accuracy: ", triplet_acc_fairness(df_fairness[df_fairness['AB_WhiteMale']==1]))
print("A and B White Male - f1 score:", f1_score(y_AB_WhiteMale, y_pred_AB_WhiteMale))
print("\n")
print("A and B Non White Male - mean accuracy: ", df_fairness[df_fairness['AB_NoWhiteMale']==1]['correct_predict'].mean())
print("A and B Non White Male - bootstrapping mean accuracy: ", bootstrap(df_fairness[df_fairness['AB_NoWhiteMale']==1], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B Non White Male - triplet accuracy: ", triplet_acc_fairness(df_fairness[df_fairness['AB_NoWhiteMale']==1]))
print("A and B Non White Male - f1 score:", f1_score(y_AB_NoWhiteMale, y_pred_AB_NoWhiteMale))
print("\n")
print("A or B Non White Male - mean accuracy: ", df_fairness[df_fairness['AB_WhiteMale']==0]['correct_predict'].mean())
print("A or B Non White Male - bootstrapping mean accuracy: ", bootstrap(df_fairness[df_fairness['AB_WhiteMale']==0], agg_func=lambda df: df['correct_predict'].mean()))
print("A or B Non White Male - triplet accuracy: ", triplet_acc_fairness(df_fairness[df_fairness['AB_WhiteMale']==0]))
print("A or B Non White Male - f1 score:", f1_score(y_AoB_NoWhiteMale, y_pred_AoB_NoWhiteMale))

In [ ]:
print("A and B White Male - Confusion Matrix")
print(confusion_matrix(y_AB_WhiteMale, y_pred_AB_WhiteMale))
print("\n")
print('A and B White Male - TPR: ', TPR_AB_WhiteMale)
print('A and B White Male - FPR: ', FPR_AB_WhiteMale)
print('A and B White Male - TNR: ', TNR_AB_WhiteMale)
print('A and B White Male - FNR: ', FNR_AB_WhiteMale)

In [ ]:
print("A and B Non White Male - Confusion Matrix")
print(confusion_matrix(y_AB_NoWhiteMale, y_pred_AB_NoWhiteMale))
print("\n")
print('A and B Non White Male - TPR: ', TPR_AB_NoWhiteMale)
print('A and B Non White Male - FPR: ', FPR_AB_NoWhiteMale)
print('A and B Non White Male - TNR: ', TNR_AB_NoWhiteMale)
print('A and B Non White Male - FNR: ', FNR_AB_NoWhiteMale)

In [ ]:
print("A or B Non White Male - Confusion Matrix")
print(confusion_matrix(y_AoB_NoWhiteMale, y_pred_AoB_NoWhiteMale))
print("\n")
print('A or B Non White Male - TPR: ', TPR_AoB_NoWhiteMale)
print('A or B Non White Male - FPR: ', FPR_AoB_NoWhiteMale)
print('A or B Non White Male - TNR: ', TNR_AoB_NoWhiteMale)
print('A or B Non White Male - FNR: ', FNR_AoB_NoWhiteMale)

#### Male vs Non Male and White vs Non White

In [ ]:
pos_dist = df_fairness[(df_fairness.y_true==1) & (df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)]['Distance']
neg_dist = df_fairness[(df_fairness.y_true==0) & (df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
pos_dist = df_fairness[(df_fairness.y_true==1) & (df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)]['Distance']
neg_dist = df_fairness[(df_fairness.y_true==0) & (df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
pos_dist = df_fairness[(df_fairness.y_true==1) & (df_fairness['A_White']==1)&(df_fairness['B_White']==1)]['Distance']
neg_dist = df_fairness[(df_fairness.y_true==0) & (df_fairness['A_White']==1)&(df_fairness['B_White']==1)]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
pos_dist = df_fairness[(df_fairness.y_true==1) & (df_fairness['A_White']==0)&(df_fairness['B_White']==0)]['Distance']
neg_dist = df_fairness[(df_fairness.y_true==0) & (df_fairness['A_White']==0)&(df_fairness['B_White']==0)]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
y_AB_Male = np.array(df_fairness[(df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)].y_true)
y_pred_AB_Male = np.array(df_fairness[(df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)].y_pred)

tp_AB_Male,fp_AB_Male,fn_AB_Male,tn_AB_Male = confusion_matrix(y_AB_Male, y_pred_AB_Male).ravel()

TPR_AB_Male = tp_AB_Male/(tp_AB_Male+fp_AB_Male)
FPR_AB_Male = fp_AB_Male/(tp_AB_Male+fp_AB_Male)
TNR_AB_Male = tn_AB_Male/(tn_AB_Male+fn_AB_Male)
FNR_AB_Male = fn_AB_Male/(tn_AB_Male+fn_AB_Male)

y_AB_NoMale = np.array(df_fairness[(df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)].y_true)
y_pred_AB_NoMale = np.array(df_fairness[(df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)].y_pred)

tp_AB_NoMale,fp_AB_NoMale,fn_AB_NoMale,tn_AB_NoMale = confusion_matrix(y_AB_NoMale, y_pred_AB_NoMale).ravel()

TPR_AB_NoMale = tp_AB_NoMale/(tp_AB_NoMale+fp_AB_NoMale)
FPR_AB_NoMale = fp_AB_NoMale/(tp_AB_NoMale+fp_AB_NoMale)
TNR_AB_NoMale = tn_AB_NoMale/(tn_AB_NoMale+fn_AB_NoMale)
FNR_AB_NoMale = fn_AB_NoMale/(tn_AB_NoMale+fn_AB_NoMale)

In [ ]:
print("A and B Male - mean accuracy: ", df_fairness[(df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)]['correct_predict'].mean())
print("A and B Male - bootstrapping mean accuracy: ", bootstrap(df_fairness[(df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B Male - triplet accuracy: ", triplet_acc_fairness(df_fairness[(df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)]))
print("A and B Male - f1 score:", f1_score(y_AB_Male, y_pred_AB_Male))
print("A and B Male - Confusion Matrix")
print(confusion_matrix(y_AB_Male, y_pred_AB_Male))
print('A and B Male - TPR: ', TPR_AB_Male)
print('A and B Male - FPR: ', FPR_AB_Male)
print('A and B Male - TNR: ', TNR_AB_Male)
print('A and B Male - FNR: ', FNR_AB_Male)
print("\n")
print("A and B Non Male - mean accuracy: ", df_fairness[(df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)]['correct_predict'].mean())
print("A and B Non Male - bootstrapping mean accuracy: ", bootstrap(df_fairness[(df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B Non Male - triplet accuracy: ", triplet_acc_fairness(df_fairness[(df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)]))
print("A and B Non Male - f1 score:", f1_score(y_AB_NoMale, y_pred_AB_NoMale))
print("A and B Non Male - Confusion Matrix")
print(confusion_matrix(y_AB_NoMale, y_pred_AB_NoMale))
print('A and B Non Male - TPR: ', TPR_AB_NoMale)
print('A and B Non Male - FPR: ', FPR_AB_NoMale)
print('A and B Non Male - TNR: ', TNR_AB_NoMale)
print('A and B Non Male - FNR: ', FNR_AB_NoMale)

### Look at images

In [ ]:
my_img = all_imgs[df_fairness[df_fairness.A_Male==0].id_A.iloc[3]]

In [ ]:
my_img = all_imgs[list(df_fairness[df_fairness.A_Male==0].id_A)]

In [ ]:
print(len(my_img))

In [ ]:
for k in range(10):
    print_img(from_tensor_to_numpy(my_img[k])/255)